In [1]:
!pip uninstall -y pyarrow datasets

Found existing installation: pyarrow 16.0.0
Uninstalling pyarrow-16.0.0:
  Successfully uninstalled pyarrow-16.0.0
Found existing installation: datasets 2.19.0
Uninstalling datasets-2.19.0:
  Successfully uninstalled datasets-2.19.0


In [2]:
!pip install --no-use-pep517 pyarrow

  Using cached pyarrow-16.0.0-cp311-cp311-manylinux_2_28_x86_64.whl.metadata (3.0 kB)
Using cached pyarrow-16.0.0-cp311-cp311-manylinux_2_28_x86_64.whl (40.8 MB)
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
mlflow 2.11.3 requires pyarrow<16,>=4.0.0, but you have pyarrow 16.0.0 which is incompatible.


In [3]:
!pip install datasets

  Using cached datasets-2.19.0-py3-none-any.whl.metadata (19 kB)
Using cached datasets-2.19.0-py3-none-any.whl (542 kB)


In [4]:
from datasets import load_dataset

dataset = load_dataset("Hello-SimpleAI/HC3", name="all")

In [5]:
print(dataset)

DatasetDict({
    train: Dataset({
        features: ['id', 'question', 'human_answers', 'chatgpt_answers', 'source'],
        num_rows: 24322
    })
})


In [6]:
!pip install torch transformers

In [7]:
human_answers = dataset['train']['human_answers']
chatgpt_answers = dataset['train']['chatgpt_answers']

In [8]:
print(human_answers[:5])
print(type(human_answers))
print(chatgpt_answers[:5])

[['Basically there are many categories of " Best Seller " . Replace " Best Seller " by something like " Oscars " and every " best seller " book is basically an " oscar - winning " book . May not have won the " Best film " , but even if you won the best director or best script , you \'re still an " oscar - winning " film . Same thing for best sellers . Also , IIRC the rankings change every week or something like that . Some you might not be best seller one week , but you may be the next week . I guess even if you do n\'t stay there for long , you still achieved the status . Hence , # 1 best seller .', "If you 're hearing about it , it 's because it was a very good or very well - publicized book ( or both ) , and almost every good or well - publicized book will be # 1 on the NY Times bestseller list for at least a little bit . Kindof like how almost every big or good movies are # 1 at the box office on their opening weekend .", "One reason is lots of catagories . However , how the NY Tim

In [9]:
from transformers import DistilBertTokenizer
from datasets import Dataset, DatasetDict

# Initialize the tokenizer
tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')

# Safely flatten the nested list structure in human_answers and chatgpt_answers
# Check if the list is non-empty and then take the first element
human_answers = [item[0] if item else "Empty response" for item in dataset['train']['human_answers']]
chatgpt_answers = [item[0] if item else "Empty response" for item in dataset['train']['chatgpt_answers']]

# Create a Hugging Face Dataset from the data (if not already in one)
data_dict = {
    "texts": human_answers + chatgpt_answers,
    "labels": [0] * len(human_answers) + [1] * len(chatgpt_answers)
}
hf_dataset = Dataset.from_dict(data_dict)

# Define the preprocessing function to tokenize the data
def preprocess_function(examples):
    # Tokenize the texts
    tokenized_inputs = tokenizer(examples['texts'], padding="max_length", truncation=True, max_length=512)
    
    # Prepare the dictionary correctly.
    return {
        'input_ids': tokenized_inputs['input_ids'], 
        'attention_mask': tokenized_inputs['attention_mask'], 
        'labels': examples['labels']
    }

# Apply the preprocessing function to the dataset
tokenized_datasets = hf_dataset.map(preprocess_function, batched=True)

# Display the first few processed entries to verify
print(tokenized_datasets.select(range(2)))  # Select the first two entries for display


Map:   0%|          | 0/48644 [00:00<?, ? examples/s]

Dataset({
    features: ['texts', 'labels', 'input_ids', 'attention_mask'],
    num_rows: 2
})


In [17]:
!pip install transformers[torch]
!pip install accelerate -U

In [18]:
from transformers import DistilBertTokenizer, DistilBertForSequenceClassification, TrainingArguments, Trainer

# Split the dataset
train_test_split = tokenized_datasets.train_test_split(test_size=0.1)  # Splitting the dataset
train_dataset = train_test_split['train']
eval_dataset = train_test_split['test']

# Load the model
model = DistilBertForSequenceClassification.from_pretrained('distilbert-base-uncased', num_labels=2)

# Training arguments
training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=3,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=16,
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir='./logs',
    logging_steps=10,
    evaluation_strategy="steps"
)

# Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset
)

# Start training
trainer.train()

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


ImportError: Using the `Trainer` with `PyTorch` requires `accelerate>=0.21.0`: Please run `pip install transformers[torch]` or `pip install accelerate -U`